In [15]:
import os
import pandas as pd
import numpy as np
from pandas.tseries.offsets import CustomBusinessDay
from pandas.tseries.holiday import USFederalHolidayCalendar
from fsf_arima_models import ArimaModels
from sklearn.metrics import mean_squared_error, mean_absolute_error
from huggingface_hub import login
from datasets import Dataset

## Get and massage portfolio data

In [2]:
ticker_histories_filename = os.path.join(
    "input", "Ticker Histories 2025-01-23 to 2025-03-20.csv"
)
ticker_histories_df = pd.read_csv(ticker_histories_filename)
ticker_histories_df["datetime"] = pd.to_datetime(
    ticker_histories_df["timestamp"], unit="ms"
).dt.tz_localize(None)
ticker_histories_df.set_index("datetime", inplace=True)
ticker_histories_df.drop("timestamp", axis=1, inplace=True)
ticker_histories_df.sort_index(inplace=True)
ticker_histories_df

,ticker,open,high,low,close,volume,vwap,transactions
datetime,,,,,,,,
2025-01-23 05:00:00,AMZN,234.100,235.5200,231.5100,235.42,25806969.0,234.2066,365072
2025-01-23 05:00:00,NVDA,145.050,147.2300,143.7200,147.22,153298302.0,145.8249,1198252
2025-01-23 05:00:00,AAPL,224.740,227.0300,222.3000,223.66,57755057.0,224.1269,619710
2025-01-23 05:00:00,MSFT,442.000,446.7500,441.5048,446.71,16975809.0,444.7685,281253
2025-01-23 05:00:00,GOOG,199.980,201.9400,196.8200,199.58,14549698.0,199.7503,211391
...,...,...,...,...,...,...,...,...
2025-03-20 04:00:00,AAPL,213.990,217.4899,212.2200,214.10,47815970.0,214.3951,499629
2025-03-20 04:00:00,GOOG,163.825,167.0300,163.1400,165.05,19326885.0,165.1973,270126
2025-03-20 04:00:00,AMZN,193.070,199.3200,192.3000,194.95,38319077.0,195.6467,455299


In [3]:
df = ticker_histories_df.reset_index().pivot(
    index="datetime", columns="ticker", values="close"
)
df.index = pd.DatetimeIndex(df.index)
df.index = pd.DatetimeIndex(
    [dt.replace(hour=17, minute=0, second=0) for dt in df.index]
)
df.sort_index(inplace=True)
df

ticker,AAPL,AMZN,GOOG,MSFT,NVDA,TSLA
2025-01-23 17:00:00,223.66,235.42,199.58,446.71,147.22,412.38
2025-01-24 17:00:00,222.78,234.85,201.90,444.06,142.62,406.58
2025-01-27 17:00:00,229.86,235.42,193.77,434.56,118.42,397.15
2025-01-28 17:00:00,238.26,238.15,197.07,447.20,128.99,398.09
2025-01-29 17:00:00,239.36,237.07,197.18,442.33,123.70,389.10
2025-01-30 17:00:00,237.59,234.64,202.63,414.99,124.65,400.28
2025-01-31 17:00:00,236.00,237.68,205.60,415.06,120.07,404.60
2025-02-03 17:00:00,228.01,237.42,202.64,410.92,116.66,383.68
2025-02-04 17:00:00,232.80,242.06,207.71,412.37,118.65,392.21
2025-02-05 17:00:00,232.47,236.17,193.30,413.29,124.83,378.17


In [4]:
def future_business_day(start_date, business_days_ahead):
    """
    Calculates the business date a specified number of business days in the future,
    skipping US federal holidays.

    Args:
        start_date (pd.Timestamp or str): The starting date.
        business_days_ahead (int): The number of business days in the future.

    Returns:
        pd.Timestamp: The calculated future business date.
    """
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays()
    cbd = CustomBusinessDay(holidays=holidays)
    return start_date + (cbd * business_days_ahead)

In [5]:
def create_business_day_range(start_date, num_days):
    """
    Creates a range of business days starting from a given date, skipping US federal holidays.

    Args:
        start_date (pd.Timestamp or str): The starting date.
        num_days (int): The number of business days to generate.

    Returns:
        pd.DatetimeIndex: A DatetimeIndex containing the range of business days.
    """
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays()
    cbd = CustomBusinessDay(holidays=holidays)
    return pd.date_range(start=start_date, periods=num_days, freq=cbd)

In [6]:
start_timestamps = create_business_day_range(pd.Timestamp("2025-01-23"), 20)
timestamp_ranges = []
for start_timestamp in start_timestamps:
    end_timestamp = future_business_day(start_timestamp, 20).replace(
        hour=23, minute=59, second=59
    )
    timestamp_ranges.append([start_timestamp, end_timestamp])
for timestamp_range in timestamp_ranges:
    print(timestamp_range)

[Timestamp('2025-01-23 00:00:00'), Timestamp('2025-02-21 23:59:59')]
[Timestamp('2025-01-24 00:00:00'), Timestamp('2025-02-24 23:59:59')]
[Timestamp('2025-01-27 00:00:00'), Timestamp('2025-02-25 23:59:59')]
[Timestamp('2025-01-28 00:00:00'), Timestamp('2025-02-26 23:59:59')]
[Timestamp('2025-01-29 00:00:00'), Timestamp('2025-02-27 23:59:59')]
[Timestamp('2025-01-30 00:00:00'), Timestamp('2025-02-28 23:59:59')]
[Timestamp('2025-01-31 00:00:00'), Timestamp('2025-03-03 23:59:59')]
[Timestamp('2025-02-03 00:00:00'), Timestamp('2025-03-04 23:59:59')]
[Timestamp('2025-02-04 00:00:00'), Timestamp('2025-03-05 23:59:59')]
[Timestamp('2025-02-05 00:00:00'), Timestamp('2025-03-06 23:59:59')]
[Timestamp('2025-02-06 00:00:00'), Timestamp('2025-03-07 23:59:59')]
[Timestamp('2025-02-07 00:00:00'), Timestamp('2025-03-10 23:59:59')]
[Timestamp('2025-02-10 00:00:00'), Timestamp('2025-03-11 23:59:59')]
[Timestamp('2025-02-11 00:00:00'), Timestamp('2025-03-12 23:59:59')]
[Timestamp('2025-02-12 00:00:00'),

In [7]:
for ticker in df.columns:
    print(ticker)

AAPL
AMZN
GOOG
MSFT
NVDA
TSLA


In [8]:
n_workers = 6
all_forecast_dfs = []
for ticker in df.columns:
    forecast_rows = []
    for start_timestamp, end_timestamp in timestamp_ranges:
        am = ArimaModels(n_workers=n_workers)
        ticker_ts = df[ticker]
        ticker_ts = ticker_ts.loc[start_timestamp:end_timestamp]
        pred_date, pred = am.fit(ticker_ts, max_p=2, max_q=2, train_len=10)
        pred_key = f"{ticker}_pred"
        pred_dict = {"pred_date": pred_date, pred_key: pred}
        print(pred_dict)
        forecast_rows.append(pred_dict)
    forecast_df = pd.DataFrame(forecast_rows).set_index("pred_date").sort_index()
    forecast_start_timestamp = forecast_df.index[0]
    forecast_end_timestamp = forecast_df.index[-1]
    forecast_df[ticker] = df.loc[
        forecast_start_timestamp:forecast_end_timestamp, ticker
    ].copy()
    print(forecast_df.head())
    all_forecast_dfs.append(forecast_df)
all_forecast_df = pd.concat(all_forecast_dfs, axis=1)

Processing: 100%|██████████| 3/3 [00:00<00:00,  5.96task/s]


1 0
{'pred_date': Timestamp('2025-02-24 17:00:00'), 'AAPL_pred': np.float64(246.39217761918874)}


/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWa

1 0
{'pred_date': Timestamp('2025-02-25 17:00:00'), 'AAPL_pred': np.float64(248.54073243492678)}


/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarn

...

/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)
Processing: 100%|██████████| 3/3 [00:00<00:00,  5.84task/s]


0 1
{'pred_date': Timestamp('2025-03-20 17:00:00'), 'TSLA_pred': np.float64(226.29486013453806)}


/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)
Processing: 100%|██████████| 3/3 [00:00<00:00,  5.78task/s]


0 1
{'pred_date': Timestamp('2025-03-21 17:00:00'), 'TSLA_pred': np.float64(228.90333256360134)}
                      TSLA_pred    TSLA
pred_date                              
2025-02-24 17:00:00  334.572302  330.53
2025-02-25 17:00:00  327.070136  302.80
2025-02-26 17:00:00  297.178568  290.80
2025-02-27 17:00:00  285.230479  281.95
2025-02-28 17:00:00  276.796647  292.98


/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency B will be used.
  self._init_dates(dates, freq)


In [9]:
all_forecast_df

,AAPL_pred,AAPL,AMZN_pred,AMZN,GOOG_pred,GOOG,MSFT_pred,MSFT,NVDA_pred,NVDA,TSLA_pred,TSLA
pred_date,,,,,,,,,,,,
2025-02-24 17:00:00,246.392178,247.10,218.127794,212.71,182.223549,181.19,407.048401,404.00,134.854098,130.28,334.572302,330.53
2025-02-25 17:00:00,248.540732,247.04,212.574953,212.80,180.115496,177.37,402.359453,397.90,132.876129,126.63,327.070136,302.80
2025-02-26 17:00:00,247.866506,240.36,211.207622,214.35,177.389939,174.70,396.129576,399.73,126.741836,131.28,297.178568,290.80
2025-02-27 17:00:00,239.737590,237.30,212.167878,208.74,174.073493,170.21,397.589698,392.53,133.664700,120.15,285.230479,281.95
2025-02-28 17:00:00,236.637939,241.84,209.188474,212.28,169.893833,172.22,391.354665,396.99,118.950744,124.92,276.796647,292.98
2025-03-03 17:00:00,242.667990,238.03,208.889637,205.02,169.119227,168.66,394.787335,388.49,124.524957,114.06,290.006670,284.65
2025-03-04 17:00:00,238.213250,235.93,205.651732,203.80,167.903006,172.61,389.822197,388.61,116.872901,115.99,279.317442,272.04
2025-03-05 17:00:00,236.145858,235.74,201.868652,208.36,168.059208,174.99,387.102323,401.02,115.415227,117.30,266.088026,279.10
2025-03-06 17:00:00,235.576146,235.33,203.514161,200.70,172.517922,174.21,396.879984,396.89,116.711043,110.57,276.276053,263.45


In [10]:
for ticker in df.columns:
    rmse = np.sqrt(
        mean_squared_error(all_forecast_df[ticker][:-1], all_forecast_df[f"{ticker}_pred"][:-1])
    )
    mae = mean_absolute_error(
        all_forecast_df[ticker][:-1], all_forecast_df[f"{ticker}_pred"][:-1]
    )
    mean_error = np.mean(all_forecast_df[ticker][:-1] - all_forecast_df[f"{ticker}_pred"][:-1])
    print(f"{ticker} forecast RMSE={rmse:.2f}, MAE={mae:.2f}, mean error={mean_error:.2f}")

AAPL forecast RMSE=4.96, MAE=3.67, mean error=-1.36
AMZN forecast RMSE=3.85, MAE=3.28, mean error=0.83
GOOG forecast RMSE=3.47, MAE=3.02, mean error=0.30
MSFT forecast RMSE=6.13, MAE=4.87, mean error=0.33
NVDA forecast RMSE=5.88, MAE=4.75, mean error=-0.42
TSLA forecast RMSE=14.59, MAE=11.88, mean error=-0.42


In [11]:
all_forecast_filename = os.path.join("output", "all_forecast.csv")
all_forecast_df.to_csv(all_forecast_filename)

In [12]:
all_forecast_df.head()

,AAPL_pred,AAPL,AMZN_pred,AMZN,GOOG_pred,GOOG,MSFT_pred,MSFT,NVDA_pred,NVDA,TSLA_pred,TSLA
pred_date,,,,,,,,,,,,
2025-02-24 17:00:00,246.392178,247.10,218.127794,212.71,182.223549,181.19,407.048401,404.00,134.854098,130.28,334.572302,330.53
2025-02-25 17:00:00,248.540732,247.04,212.574953,212.80,180.115496,177.37,402.359453,397.90,132.876129,126.63,327.070136,302.80
2025-02-26 17:00:00,247.866506,240.36,211.207622,214.35,177.389939,174.70,396.129576,399.73,126.741836,131.28,297.178568,290.80
2025-02-27 17:00:00,239.737590,237.30,212.167878,208.74,174.073493,170.21,397.589698,392.53,133.664700,120.15,285.230479,281.95
2025-02-28 17:00:00,236.637939,241.84,209.188474,212.28,169.893833,172.22,391.354665,396.99,118.950744,124.92,276.796647,292.98


In [13]:
all_forecast_df.tail()

,AAPL_pred,AAPL,AMZN_pred,AMZN,GOOG_pred,GOOG,MSFT_pred,MSFT,NVDA_pred,NVDA,TSLA_pred,TSLA
pred_date,,,,,,,,,,,,
2025-03-17 17:00:00,212.319161,214.00,190.636395,195.74,165.569698,166.57,384.741735,388.70,117.915222,119.53,243.121678,238.01
2025-03-18 17:00:00,212.652952,212.69,192.774397,192.82,165.638440,162.67,387.560683,383.52,119.141665,115.43,234.647067,225.31
2025-03-19 17:00:00,211.215589,215.24,191.393277,195.54,162.325617,166.28,382.889509,387.82,115.590403,117.52,222.662252,235.86
2025-03-20 17:00:00,213.887546,214.10,191.870942,194.95,163.736341,165.05,385.153116,386.84,115.113280,118.53,226.294860,236.26
2025-03-21 17:00:00,212.628780,NaN,191.968117,NaN,164.157765,NaN,385.399745,NaN,116.818549,NaN,228.903333,NaN


In [14]:
hf_token = os.getenv("HF_TOKEN")
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [16]:
dataset = Dataset.from_pandas(all_forecast_df)
dataset.push_to_hub("akey7/fsf-ticker-preds-vs-actuals")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/akey7/fsf-ticker-preds-vs-actuals/commit/55ce654aa71f59608ed70a6602b018f2e0a3bc17', commit_message='Upload dataset', commit_description='', oid='55ce654aa71f59608ed70a6602b018f2e0a3bc17', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/akey7/fsf-ticker-preds-vs-actuals', endpoint='https://huggingface.co', repo_type='dataset', repo_id='akey7/fsf-ticker-preds-vs-actuals'), pr_revision=None, pr_num=None)